Padding and Stride

6.3.1. Lapisan

Dalam contoh berikut, kami membuat lapisan konvolusi dua dimensi dengan tinggi dan lebar 3 dan menerapkan 1 piksel padding di semua sisi. Diberikan input dengan tinggi dan lebar 8, kami menemukan bahwa tinggi dan lebar output juga 8.

In [1]:
import torch
from torch import nn


# We define a convenience function to calculate the convolutional layer. This
# function initializes the convolutional layer weights and performs
# corresponding dimensionality elevations and reductions on the input and
# output
def comp_conv2d(conv2d, X):
    # Here (1, 1) indicates that the batch size and the number of channels
    # are both 1
    X = X.reshape((1, 1) + X.shape)
    Y = conv2d(X)
    # Exclude the first two dimensions that do not interest us: examples and
    # channels
    return Y.reshape(Y.shape[2:])
# Note that here 1 row or column is padded on either side, so a total of 2
# rows or columns are added
conv2d = nn.Conv2d(1, 1, kernel_size=3, padding=1)
X = torch.rand(size=(8, 8))
comp_conv2d(conv2d, X).shape

torch.Size([8, 8])

Ketika tinggi dan lebar kernel konvolusi berbeda, kita dapat membuat output dan input memiliki tinggi dan lebar yang sama dengan mengatur nomor padding yang berbeda untuk tinggi dan lebar.

In [2]:
# padding numbers on either side of the height and width are 2 and 1,
# respectively
conv2d = nn.Conv2d(1, 1, kernel_size=(5, 3), padding=(2, 1))
comp_conv2d(conv2d, X).shape

torch.Size([8, 8])

6.3.2. Melangkah

Di bawah, kami mengatur langkah pada tinggi dan lebar menjadi 2, sehingga mengurangi separuh tinggi dan lebar input.

In [3]:
conv2d = nn.Conv2d(1, 1, kernel_size=3, padding=1, stride=2)
comp_conv2d(conv2d, X).shape

torch.Size([4, 4])

In [4]:
#Next, we will look at a slightly more complicated example.
conv2d = nn.Conv2d(1, 1, kernel_size=(3, 5), padding=(0, 1), stride=(3, 4))
comp_conv2d(conv2d, X).shape

torch.Size([2, 2])

6.3.3. Ringkasan

Padding dapat meningkatkan tinggi dan lebar output. Ini sering digunakan untuk memberikan output dengan tinggi dan lebar yang sama dengan input.

Langkahnya dapat mengurangi resolusi output, misalnya mengurangi tinggi dan lebar output menjadi hanya 1/n dari tinggi dan lebar input ( n adalah bilangan bulat yang lebih besar dari 1 ).

Padding dan stride dapat digunakan untuk menyesuaikan dimensi data secara efektif.

6.4. Multiple Input and Multiple Output Channels

6.4.1. Beberapa Saluran Masukan

Untuk memastikan kami benar-benar memahami apa yang terjadi di sini, kami dapat menerapkan operasi korelasi silang dengan beberapa saluran input sendiri. Perhatikan bahwa semua yang kita lakukan adalah melakukan satu operasi korelasi silang per saluran dan kemudian menjumlahkan hasilnya.

In [5]:
!pip install d2l

     |████████████████████████████████| 82 kB 472 kB/s 
     |████████████████████████████████| 9.9 MB 35.9 MB/s 
     |████████████████████████████████| 61 kB 6.4 MB/s 
     |████████████████████████████████| 11.6 MB 43.3 MB/s 
     |████████████████████████████████| 20.1 MB 1.3 MB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: pandas
    Found existing installation: pandas 1.1.5
    Uninstalling pandas-1.1.5:
      Successfully uninstalled pandas-1.1.5
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.2.2
    Uninstalling matplotlib-3.2.2:
      Successfully uninstalled matplotlib-3.2.2
ERROR: pip's dependency resolver does not currently take into account all the p

In [6]:
import torch
from d2l import torch as d2l

def corr2d_multi_in(X, K):
    # First, iterate through the 0th dimension (channel dimension) of `X` and
    # `K`. Then, add them together
    return sum(d2l.corr2d(x, k) for x, k in zip(X, K))

In [7]:
X = torch.tensor([[[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]],
               [[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]]])
K = torch.tensor([[[0.0, 1.0], [2.0, 3.0]], [[1.0, 2.0], [3.0, 4.0]]])

corr2d_multi_in(X, K)

tensor([[ 56.,  72.],
        [104., 120.]])

6.4.2. Multiple Output Channels

Kami menerapkan fungsi korelasi silang untuk menghitung output dari beberapa saluran seperti yang ditunjukkan di bawah ini.

In [8]:
def corr2d_multi_in_out(X, K):
    # Iterate through the 0th dimension of `K`, and each time, perform
    # cross-correlation operations with input `X`. All of the results are
    # stacked together
    return torch.stack([corr2d_multi_in(X, k) for k in K], 0)

In [9]:
K = torch.stack((K, K + 1, K + 2), 0)
K.shape

torch.Size([3, 2, 2, 2])

In [10]:
corr2d_multi_in_out(X, K)

tensor([[[ 56.,  72.],
         [104., 120.]],

        [[ 76., 100.],
         [148., 172.]],

        [[ 96., 128.],
         [192., 224.]]])

6.4.3. 1×1 Convolutional Layer

Mari kita periksa apakah ini berhasil dalam praktiknya: kita menerapkan konvolusi 1x1 menggunakan lapisan yang terhubung penuh. Satu-satunya hal adalah kita perlu melakukan beberapa penyesuaian pada bentuk data sebelum dan sesudah perkalian matriks.

In [11]:
def corr2d_multi_in_out_1x1(X, K):
    c_i, h, w = X.shape
    c_o = K.shape[0]
    X = X.reshape((c_i, h * w))
    K = K.reshape((c_o, c_i))
    # Matrix multiplication in the fully-connected layer
    Y = torch.matmul(K, X)
    return Y.reshape((c_o, h, w))

In [12]:
X = torch.normal(0, 1, (3, 3, 3))
K = torch.normal(0, 1, (2, 3, 1, 1))

Y1 = corr2d_multi_in_out_1x1(X, K)
Y2 = corr2d_multi_in_out(X, K)
assert float(torch.abs(Y1 - Y2).sum()) < 1e-6

6.5. Pooling

6.5.1. Maximum Pooling and Average Pooling

Dalam kode di bawah ini, kami menerapkan propagasi maju dari lapisan penyatuan dalam fungsi pool2d. Fungsi ini mirip dengan fungsi corr2d di Bagian 6.2. Namun, di sini kami tidak memiliki kernel, yang menghitung output sebagai maksimum atau rata-rata setiap wilayah dalam input.

In [13]:
import torch
from torch import nn
from d2l import torch as d2l

def pool2d(X, pool_size, mode='max'):
    p_h, p_w = pool_size
    Y = torch.zeros((X.shape[0] - p_h + 1, X.shape[1] - p_w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            if mode == 'max':
                Y[i, j] = X[i: i + p_h, j: j + p_w].max()
            elif mode == 'avg':
                Y[i, j] = X[i: i + p_h, j: j + p_w].mean()
    return Y

In [14]:
X = torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
pool2d(X, (2, 2))

tensor([[4., 5.],
        [7., 8.]])

In [15]:
pool2d(X, (2, 2), 'avg')

tensor([[2., 3.],
        [5., 6.]])

6.5.2. Padding and Stride

As with convolutional layers, pooling layers can also change the output shape.And as before, we can alter the operation to achieve a desired output shape by padding the input and adjusting the stride.

In [16]:
X = torch.arange(16, dtype=torch.float32).reshape((1, 1, 4, 4))
X

tensor([[[[ 0.,  1.,  2.,  3.],
          [ 4.,  5.,  6.,  7.],
          [ 8.,  9., 10., 11.],
          [12., 13., 14., 15.]]]])

Secara default, stride dan pooling window dalam instance dari kelas bawaan framework memiliki bentuk yang sama. Di bawah ini, kita menggunakan pooling window of shape (3, 3), jadi kita mendapatkan bentuk stride (3, 3) secara default.

In [17]:
pool2d = nn.MaxPool2d(3)
pool2d(X)

#The stride and padding can be manually specified.

pool2d = nn.MaxPool2d(3, padding=1, stride=2)
pool2d(X)

#Of course, we can specify an arbitrary rectangular pooling window and specify the padding and stride for height and width, respectively.

pool2d = nn.MaxPool2d((2, 3), stride=(2, 3), padding=(0, 1))
pool2d(X)

tensor([[[[ 5.,  7.],
          [13., 15.]]]])

6.5.3. Multiple Channels

Saat memproses data input multi-saluran, lapisan penyatuan mengumpulkan setiap saluran input secara terpisah, daripada menjumlahkan input di atas saluran seperti pada lapisan konvolusi. Ini berarti bahwa jumlah saluran keluaran untuk lapisan penyatuan sama dengan jumlah saluran masukan.

Di bawah ini, kami akan menggabungkan tensor X dan X + 1 pada dimensi saluran untuk membuat input dengan 2 saluran.

In [18]:
X = torch.cat((X, X + 1), 1)
X

tensor([[[[ 0.,  1.,  2.,  3.],
          [ 4.,  5.,  6.,  7.],
          [ 8.,  9., 10., 11.],
          [12., 13., 14., 15.]],

         [[ 1.,  2.,  3.,  4.],
          [ 5.,  6.,  7.,  8.],
          [ 9., 10., 11., 12.],
          [13., 14., 15., 16.]]]])

In [19]:
pool2d = nn.MaxPool2d(3, padding=1, stride=2)
pool2d(X)

tensor([[[[ 5.,  7.],
          [13., 15.]],

         [[ 6.,  8.],
          [14., 16.]]]])